In [49]:
try:
    import efficient_apriori
except:
    !pip install efficient_apriori

  Using cached efficient_apriori-2.0.1-py3-none-any.whl (14 kB)


In [55]:
import pandas as pd
import numpy as np

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

from efficient_apriori import apriori as apriori1

In [127]:
data = pd.read_csv("Groceries_dataset.csv")
data.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [128]:
array = []
for member in data['Member_number'].unique():
    unique_member = data.loc[data['Member_number'] == member]
    for day in unique_member['Date'].unique():
        same_day = unique_member.loc[unique_member['Date'] == day]
        array.append(list(same_day['itemDescription']))
        
        

In [129]:
array[:10]

[['tropical fruit', 'rolls/buns', 'candy'],
 ['long life bakery product', 'semi-finished bread'],
 ['meat', 'whole milk'],
 ['sugar', 'citrus fruit', 'napkins'],
 ['whole milk', 'tropical fruit', 'chocolate'],
 ['butter', 'root vegetables'],
 ['female sanitary products', 'whole milk', 'hygiene articles'],
 ['pot plants', 'shopping bags', 'coffee'],
 ['other vegetables', 'chocolate'],
 ['pip fruit', 'other vegetables', 'flour']]

# Using mxtendl


In [102]:
tre = TransactionEncoder()
tre_arr = tre.fit(array).transform(array)
df = pd.DataFrame(tre_arr, columns = tre.columns_)

In [170]:
from mlxtend.frequent_patterns import apriori

detail = apriori(df, min_support=0.0003,use_colnames=True)
#detail.sort_values('support',ascending = False)[:10]

In [171]:
detail['length'] = detail['itemsets'].apply(lambda x: len(x))

In [172]:
detail[(detail['itemsets'].apply(lambda x: 'milk' in str(x)))
       & (detail['length'] == 1)]

,support,itemsets,length
1,0.021386,(UHT-milk),1
14,0.017577,(butter milk),1
33,0.006549,(condensed milk),1
160,0.157923,(whole milk),1


In [199]:
detail[(detail['itemsets'].apply(lambda x: 'fruit' in str(x)))
       & (detail['length'] == 1)]

,support,itemsets,length
20,0.001403,(canned fruit),1
28,0.053131,(citrus fruit),1
58,0.000735,(frozen fruits),1
62,0.034017,(fruit/vegetable juice),1
100,0.008488,(packaged fruit/vegetables),1
106,0.049054,(pip fruit),1
152,0.067767,(tropical fruit),1


In [193]:
detail['length'] = detail['itemsets'].apply(lambda x: len(x))
detail[(detail['length'] >= 2)].sort_values('support',ascending = False)[:10]

,support,itemsets,length
1723,0.014837,"(other vegetables, whole milk)",2
1888,0.013968,"(rolls/buns, whole milk)",2
1994,0.011629,"(soda, whole milk)",2
2046,0.011161,"(yogurt, whole milk)",2
1698,0.010559,"(rolls/buns, other vegetables)",2
1707,0.009691,"(soda, other vegetables)",2
1945,0.008955,"(sausage, whole milk)",2
2031,0.008220,"(tropical fruit, whole milk)",2
1724,0.008087,"(yogurt, other vegetables)",2
1874,0.008087,"(rolls/buns, soda)",2


# Using efficient_apriori

In [197]:
itemsets, rules = apriori1(array, min_support=0.001, min_confidence=0.1,output_transaction_ids=True)
results = list(rules)

In [198]:
rules_rhs = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.support, reverse = True):
  print(rule)

{whole milk, yogurt} -> {sausage} (conf: 0.132, supp: 0.001, lift: 2.183, conv: 1.082)
{sausage, yogurt} -> {whole milk} (conf: 0.256, supp: 0.001, lift: 1.620, conv: 1.132)
{sausage, whole milk} -> {yogurt} (conf: 0.164, supp: 0.001, lift: 1.912, conv: 1.094)
{whole milk, yogurt} -> {rolls/buns} (conf: 0.120, supp: 0.001, lift: 1.089, conv: 1.011)
{rolls/buns, yogurt} -> {whole milk} (conf: 0.171, supp: 0.001, lift: 1.082, conv: 1.016)
{other vegetables, rolls/buns} -> {whole milk} (conf: 0.114, supp: 0.001, lift: 0.721, conv: 0.950)
{rolls/buns, soda} -> {other vegetables} (conf: 0.140, supp: 0.001, lift: 1.151, conv: 1.021)
{other vegetables, soda} -> {rolls/buns} (conf: 0.117, supp: 0.001, lift: 1.066, conv: 1.008)
{other vegetables, rolls/buns} -> {soda} (conf: 0.108, supp: 0.001, lift: 1.108, conv: 1.012)
{other vegetables, soda} -> {whole milk} (conf: 0.117, supp: 0.001, lift: 0.742, conv: 0.954)
{whole milk, yogurt} -> {other vegetables} (conf: 0.102, supp: 0.001, lift: 0.834, 

In [201]:
itemsets, rules = apriori1(array, min_support=0.0003, min_confidence=0.01,output_transaction_ids=True)
results = list(rules)

In [202]:
rules_rhs = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 2, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.support, reverse = True):
  print(rule)

{soda, whole milk} -> {other vegetables, pastry} (conf: 0.029, supp: 0.000, lift: 7.818, conv: 1.026)
{pastry, whole milk} -> {other vegetables, soda} (conf: 0.052, supp: 0.000, lift: 5.319, conv: 1.044)
{pastry, soda} -> {other vegetables, whole milk} (conf: 0.082, supp: 0.000, lift: 5.525, conv: 1.073)
{other vegetables, whole milk} -> {pastry, soda} (conf: 0.023, supp: 0.000, lift: 5.525, conv: 1.019)
{other vegetables, soda} -> {pastry, whole milk} (conf: 0.034, supp: 0.000, lift: 5.319, conv: 1.029)
{other vegetables, pastry} -> {soda, whole milk} (conf: 0.091, supp: 0.000, lift: 7.818, conv: 1.087)
{whole milk, yogurt} -> {rolls/buns, sausage} (conf: 0.030, supp: 0.000, lift: 5.600, conv: 1.025)
{sausage, yogurt} -> {rolls/buns, whole milk} (conf: 0.058, supp: 0.000, lift: 4.162, conv: 1.047)
{sausage, whole milk} -> {rolls/buns, yogurt} (conf: 0.037, supp: 0.000, lift: 4.772, conv: 1.031)
{rolls/buns, yogurt} -> {sausage, whole milk} (conf: 0.043, supp: 0.000, lift: 4.772, conv: